# Step37, 텐서를 다루다

지금까지 주로 '스칼라'를 다뤘다.  
머신러닝 데이터로는 벡터나 행렬 등의 '텐서'가 주로 쓰인다.

이번 단계에서는 텐서를 사용할 때의 주의점을 알아보면서 DeZero확장을 준비한다.  
또한 지금까지 구현한 DeZero 함수들이 텐서도 문제없이 다룰 수 있음을 보여준다.

## 37.1 원소별 계산

add, mul, div, sin등의 DeZero 함수를 구현했다.  
이 함수들을 구현하면서 '입력과 출력'이 모두 '스칼라'라고 가정했다.

x가 단일값인 스칼라(0차원의 ndarray 인스턴스)인 경우

In [1]:
if '__file__' in globals():
    import os, sys 
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

import numpy as np 
import dezero.functions as F 
from dezero import Variable

x = Variable(np.array(1.0))
y = F.sin(x)
print(y)

variable(0.8414709848078965)


x가 텐서일 경우  
sin 함수가 x의 원소 각각에 적용된다.  즉, 입력과 출력 테서의 형상은 바뀌지 않는다.

In [2]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sin(x)
print(y)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])


In [3]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
y = x + c
print(y)

variable([[11 22 33]
          [44 55 66]])


## 37.2 텐서 사용시의 역전파 

지금까지의 역전파 구현은 '스칼라'를 대상으로 했다.

'텐서'를 이용해 계산해도 역전파 코드가 문제없이 작동한다.
- 그동안 '스칼라'를 대상으로 역전파를 구현했다.
- DeZero 함수에 '텐서'를 건네면 텐서의 원소마다 '스칼라'로 계산한다.
- 텐서의 원소별 '스칼라' 계산이 이루어지면 '스칼라'를 가정해 구현한 역전파는 '텐서'의 원소별 계산에서도 성립

위 논리로부터 원소별 계산을 수행하는 DeZero 함수들은 '텐서'를 사용한 계산에도 역전파를 올바르게 해낼 것임을 유추할 수 있다.

In [4]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
t = x + c 
y = F.sum(t)
print(y)

variable(231)


sum 함수를 사용하면 주어진 텐서에의 모든 원소의 총합을 구해 하나의 스칼라로 출력한다.  
그래서 앞의 코드에서 x,c,t는 모두 (2,3)이지만 마지막 출력 y는 스칼라이다.

**NOTE_** 머신러닝 문제에서는 텐서를 입력받아 스칼라를 출력하는 함수(손실 함수<sup>loss function</sup>)를 설정하는 것이 일반적이다.

In [5]:
y.backward(retain_grad=True)    # 각 변수의 미분값이 구해진다., True로 설정했으므로 미분값이 유지된다.
print(y.grad)       # 스칼라
print(t.grad)       # 텐서끼리 더해 텐서가 나옴
print(x.grad)       # 텐서 그 자체 
print(c.grad)       # 텐서 그 자체

variable(1)
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])


지금까지 구현한 함수들은 텐서를 사용한 계산에서도 제대로 역전파할 수 있다.  

여기서 중요한 특징  
기울기의 shape과 데이터(순전파때의 데이터)의 shape이 일치한다는 것.  
즉, x.shape == x.grad.shape  
c.shape == c.grad.shape  
t.shape == t.grad.shape이다. 

이 성질을 이용하면 원소별 계산이 아닌 함수, 가령 sum이나 reshape 함수등을 구현하기 어렵지 않다.